In [36]:
import pandas as pd
import csv
df = pd.read_csv('file.csv')

df_qas = df.iloc[:,[6,7,8]]
df_chatgpt = df.iloc[:,[6]]['Q&A by ChatGPT']
df_gemini = df.iloc[:,[7]]['Q&A by GEMINI']
df_chatgpt_answer = df.iloc[:,[8]]['answers']

In [37]:
df_qas = df_qas.dropna()
df_chatgpt = df_chatgpt.dropna()
df_gemini = df_gemini.dropna()
df_chatgpt_answer = df_chatgpt_answer.dropna()

In [38]:
df_context = df.iloc[:,[5]]['STATEMENT']
df_context = df_context.dropna()
df_context

1     3.2. Kerangka Kebijakan Makro Pembangunan\nPer...
6     Kerangka Kebijakan Agenda Pembangunan\nKerangk...
7     Agenda Pembangunan 1: Memperkuat Ketahanan Eko...
8     Pemenuhan Kebutuhan Energi dengan Mengutamakan...
9     Peningkatan Kuantitas/Ketahanan Air untuk Mend...
10    Peningkatan Ketersediaan, Akses, dan Kualitas ...
11    Peningkatan Pengolahan Kemaritiman, Perikanan,...
12    Penguatan Kewirausahaan, UMKM, dan Koperasi\nA...
13    Peningkatan Nilai Tambah, Lapangan Kerja dan I...
14    Peningkatan Ekspor Bernilai Tambah Tinggi dan ...
15    Penguatan Pilar Pertumbuhan Daya Saing\nDalam ...
16    Kebijakan pengembangan wilayah untuk mengurang...
17    Agenda Pembangunan 3: Meningkatkan Sumber Daya...
18    Agenda Pembangunan 4: Revolusi Mental dan Pemb...
19    Agenda Pembangunan 5: Memperkuat Infrastruktur...
20    Agenda Pembangunan 6: Membangun Lingkungan Hid...
21    Agenda Pembangunan 7: Memperkuat Stabilitas Po...
Name: STATEMENT, dtype: object

In [39]:
def get_text_between_tanda(text, tanda):
    temp = []
    for i in text:
        text = i.split(tanda)
        temp.append(text)
    return temp


def looping_data(df):
    temp = []
    for i in range(len(df)):
        temp.append(df.iloc[i])
    return temp

In [40]:
def hapus_item_genap(arr):
    return [item for i, item in enumerate(arr) if i % 2 != 0]

def hapus_newline(arr):
    return [item.replace('\n', ' ') for item in arr]
def hapus_newline_r(arr):
    return [item.replace('\r', ' ') for item in arr]
def hapus_bintang(arr):
    return [item.replace('*', ' ') for item in arr]

In [41]:
def find_answer_indices(context, answer_text):
    """
    Fungsi untuk menemukan indeks posisi awal dan akhir dari teks jawaban dalam konteks.

    Parameters:
    context (str): Teks konteks di mana jawaban akan dicari.
    answer_text (str): Teks jawaban yang akan dicari dalam konteks.

    Returns:
    tuple: Posisi indeks awal dan akhir dari teks jawaban dalam konteks. Jika jawaban tidak ditemukan, mengembalikan (-1, -1).
    """
    start_index = context.find(answer_text)
    if start_index == -1:
        return -1, -1  # Jawaban tidak ditemukan dalam konteks
    end_index = start_index + len(answer_text)
    return start_index, end_index

In [42]:
data_gemini = looping_data(df_gemini)
split_gemini = get_text_between_tanda(data_gemini, "**")
gemini_QAS = []
for i in range(len(split_gemini)):
    split_gemini[i].pop(0)
    hasil = hapus_item_genap(split_gemini[i])
    hasil = hapus_bintang(hasil)
    hasil = hapus_newline(hasil)
    hasil = hapus_newline_r(hasil)
    gemini_QAS.append([hasil, len(hasil)])

pertanyaan = []
jawaban = []
for i in range(len(gemini_QAS)):
    temp_pertanyaan = []
    temp_jawaban = []
    for j in range(gemini_QAS[i][1]):
        if(j % 2 == 0):
            temp_pertanyaan.append(gemini_QAS[i][0][j])
        else:
            temp_jawaban.append(gemini_QAS[i][0][j])
    pertanyaan.append(temp_pertanyaan)
    jawaban.append(temp_jawaban)

context = []
for idx, x in enumerate(df_context):
    context.append(x)

qas = []
id = 0
for i in range(len(pertanyaan)):
    qas_temp = []
    for j in range(len(pertanyaan[i])):
        start_index, end_index = find_answer_indices(context[i], jawaban[i][j])
        dict_list = {} 
        dict_list["id"] = id
        dict_list["title"] = "Evaluasi Paruh Waktu RPJMN 2020-2024"
        dict_list["context"] = context[i]
        dict_list["question"] = pertanyaan[i][j]
        dict_list["answer"] = jawaban[i][j]
        qas_temp.append(dict_list)
        id += 1
    qas.append(qas_temp)

In [43]:
data_gpt = looping_data(df_chatgpt)
split_gpt = get_text_between_tanda(data_gpt, "**")

gpt_QAS = []
for i in range(len(split_gpt)):
  split_gpt[i].pop(0)
  hasil = hapus_item_genap(split_gpt[i])
  hasil = hapus_bintang(hasil)
  hasil = hapus_newline(hasil)
  hasil = hapus_newline_r(hasil)

  gpt_QAS.append([hasil, len(hasil)])
# print(gpt_QAS)
pertanyaan_gpt = []
jawaban_gpt = []

for i in range(len(gpt_QAS)):
  temp_pertanyaan = []
  temp_jawaban = []
  for j in range(gpt_QAS[i][1]):
    if(j % 2 == 0):
      temp_pertanyaan.append(gpt_QAS[i][0][j])
    else:
      temp_jawaban.append(gpt_QAS[i][0][j])
  pertanyaan_gpt.append(temp_pertanyaan)
  jawaban_gpt.append(temp_jawaban)

for i in range(len(pertanyaan_gpt)):
  qas_temp = []
  for j in range(len(pertanyaan_gpt[i])):
    # print(pertanyaan_gpt[i][j])
    dict_list = {}
    dict_list['id'] = id
    dict_list['title'] = "Evaluasi Paruh Waktu RPJMN 2020-2024"
    dict_list['context'] = context[i]
    dict_list['question'] = pertanyaan_gpt[i][j]
    dict_list['answer'] = jawaban_gpt[i][j]
    qas_temp.append(dict_list)
    id += 1
  qas.append(qas_temp)

In [44]:
# print(qas[0])

In [46]:
# konteks = []
# pertanyaan = []

# for i in qas:
#   konteks.append(i[0]['context'])
#   for j in i:
#     pertanyaan.append(j['question'])

# # to csv
# dict_list_konteks = []
# dict_list_pertanyaan = []
# for i in range(len(konteks)):
#   dict_list_konteks.append({'Konteks': konteks[i]})

# for i in range(len(pertanyaan)):
#   dict_list_pertanyaan.append({'Pertanyaan': pertanyaan[i]})

# df_k = pd.DataFrame(dict_list_konteks)
# df_k.to_csv('data_k.csv', index=False)

# df_p = pd.DataFrame(dict_list_pertanyaan)
# df_p.to_csv('data_p.csv', index=False)

In [59]:
jsonl = []
for i in qas:
  for j in i:
    if(len(j['question']) > 10):
      jsonl.append(j)

In [48]:
# # covert to xlsx
# df = pd.DataFrame(jsonl)
# df.to_excel('output.xlsx', index=False)
# print("Berhasil convert to xlsx")

In [60]:
# convert to csv
df = pd.DataFrame(jsonl)
df.to_csv('output.csv', index=False)


In [50]:
# # convert array to jsonl
# import json

# with open('data.jsonl', 'w') as outfile:
#     for entry in jsonl:
#         json.dump(entry, outfile)
#         outfile.write('\n')